### 교차검증(cross validation)

* 검증 데이터: 훈련데이터에서 모델을 검증하기 위해 떼어 놓는 데이터
    * 실전에서 테스트는 모델을 성능 평가에 사용할 수 없기 때문에
* 교차 검증: 데이터가 적을 때 모델을 그나마(?) 안정적으로 평가할 수 있는 방법
    * 훈련 데이터로 모델을 학습 시키고
    * 검증 데이터로 모델을 평가
    * 모델의 최종 평가는 검증 데이터의 평균으로!
        |||||||
        |:---:|:---:|:---:|:---:|:---:|:---:|
        |실험1|**검증**|훈련|훈련|훈련|훈련|
        |실험2|훈련|**검증**|훈련|훈련|훈련|
        |실험3|훈련|훈련|**검증**|훈련|훈련|
        |실험4|훈련|훈련|훈련|**검증**|훈련|
        |실험5|훈련|훈련|훈련|훈련|**검증**|

* $k$-fold cross validation: 훈련 데이터를 $k$등분 하여 교차 검증하는 방법
    * $k$가 5(20%) 혹은 10(10%)일때를 많이 사용
    
**주의! `cross_validate()`는 훈련 데이터를 섞어주지는 않는다!**

**따라서 분류 모델을 만들 때는 `cv=StratifiedKFold()`를 지정해 주는 것이 좋다.**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats("png2x") # svg, retina, png2x ...
mpl.style.use("seaborn-v0_8")
mpl.rcParams.update({"figure.constrained_layout.use": True})
sns.set_context("paper") 
sns.set_palette("Set2") 
sns.set_style("whitegrid") 

# 시스템 폰트패밀리에 따라 변경
plt.rc("font", family = "NanumSquareRound")
plt.rcParams["axes.unicode_minus"] = False

In [50]:
heart = pd.read_csv("../../data/heart.csv")

X = heart[[i for i in heart.columns if i != "age"]]
y = heart["age"]

In [51]:
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.20,
                                                    random_state = 42)

In [52]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

model = DecisionTreeClassifier(max_depth = 6, random_state=42) 

In [53]:
splitter = StratifiedKFold(n_splits = 5, shuffle=True, random_state = 42)

score_strat = cross_validate(estimator = model, 
                             X = x_train, 
                             y = y_train, 
                             cv = splitter)# or StratifiedKFold()
np.mean(score_strat["test_score"])
score_strat

/home/tommy/miniconda3/envs/p311/lib/python3.11/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


{'fit_time': array([0.00219345, 0.00246954, 0.0020268 , 0.00324678, 0.00295925]),
 'score_time': array([0.00104856, 0.00120163, 0.000947  , 0.00168705, 0.00173974]),
 'test_score': array([0.06122449, 0.10204082, 0.02083333, 0.02083333, 0.04166667])}

In [54]:
model.fit(x_train.values, y_train.values)
prediction = model.predict(x_test.values)
print('The accuracy of the DT is', 
      accuracy_score(prediction, y_test))

The accuracy of the DT is 0.06557377049180328
